In [1]:
from tqdm import tqdm
tqdm.pandas()
import re
import os
import pandas as pd
import pickle
import time
from datetime import datetime
import bson
import pymongo
from unidecode import unidecode
from collections import Counter

/home/ubuntu/anaconda3/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import pymongo
import warnings
warnings.filterwarnings("ignore")
username = "tagger-admin"
password = "tvaiadmin"
db_client = pymongo.MongoClient('104.198.62.226', username=username,
                                password=password,
                                authSource='tags',
                                authMechanism='SCRAM-SHA-256', port=27017).tags

In [19]:
db_client["tweets_pipeline_v2"].find({"status": "daily-shot",
#                                       'source': 'dailyshot'
                                     }).count()

897

In [5]:
db_client["tweets_pipeline_v2"].find({"status": "pipelined",
#                                       "weakTag": True,
                                      'source': 'dailyshot'}).count()

882

In [16]:
db_client["tweets_pipeline_v2"].find({
                                    "status": "pipelined",
                                      "username": "soberlook",
                                      "source": "twitter", 
                                     }).count()

15

In [45]:
statuses = []
for i in db_client["tweets_pipeline_v2"].find(
    {"$or": [
            {"status":"validated", "source": "dailyshot"},
            {"status":"validated", "source": "twitter", "username": "soberlook"},
        ]},
                                        ):
    print(i)
#     statuses.append(i["status"])

{'_id': ObjectId('5e8f6f9b245baf88bed3afbd'), 'tweet_text': 'RT @GregDaco: Global crude oil demand has plunged 20%\n\nvia @WSJ @SoberLook https://t.co/299Lkx5J1p', 'hashtags': [], 'username': 'soberlook', 'tweet_timestamp': datetime.datetime(2020, 4, 6, 18, 9, 56), 'tweet_id': 1247225039885385731, 'img_urls': 'https://pbs.twimg.com/media/EU6z_uwWkAISjwT.png', 'TIME_FIELD_FOR_SORTING': datetime.datetime(2020, 4, 6, 18, 9, 56), 'source': 'twitter', 'img_type': 'graphic', 'tech_type': 'norm', 'status': 'validated', 'extracted_text': 'Global crude-oil demand, quarterly Posted on 105 million barrels a day WSJ: The Daily Shot 06-Apr-2020 90 75 mmm..rim mimy 1990 "95 2000 "05 10 \'15 \'20 Note: Figures for 2020 are estimates.', 'bert_tags': ['commodities', 'energy', 'global'], 'industries_from_bert': ['Energy sources'], 'shifts_from_bert': ['Clean energy'], 'trasher': [], 'pipelined_ts': datetime.datetime(2020, 4, 9, 18, 55, 23, 628000), 'pipelined_issued': True, 'confirmed_after_validate': [

In [42]:
'validated',
'pipelined'

'pipelined'

In [43]:
Counter(statuses)

Counter({'validated': 6})

In [ ]:
'trasher', 
# 'pipelined', 
"validated", 
"deleted", 
'deleted_from_analytics', 
# "graphicone_feed", 
# 'graphicone_search'

In [46]:
def updateAPP(bson_id):
    date = datetime.now()        
    
    before = db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})

    industries_from_bert = before["industries_from_bert"]
    grafeed_industries = industries_from_bert
    
    shifts_from_bert = before["shifts_from_bert"]
    grafeed_shifts = shifts_from_bert
    
    grafeed_confirmed = list(set(before["bert_tags"] + before["entity_prepared"]))
#     grafeed_confirmed = before["confirmed_after_validate"]

#     source = before["previous_source"]
    
    
#     print(before["bert_tags"] + before["entity_prepared"])
#     print(grafeed_confirmed)
    
#     print(industries_from_bert)
#     print(grafeed_industries)
    
#     print(shifts_from_bert)
#     print(grafeed_shifts)
    
    
#     print()
#     print()


    db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(bson_id)},
                                    {"$set":{"status": "daily-shot",
                                             "grafeed_industries": grafeed_industries,
                                             "grafeed_shifts": grafeed_shifts,
                                             "grafeed_confirmed": grafeed_confirmed,
#                                              "source": source,                                             
                                             "approved_timestamp": date,
                                             "additional_info": "moderator tool passed",
                                             "grafeed_approver": "bond",
                                            }})
    
    

    after = db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})
    print("AFTER:", after["status"], after["approved_timestamp"], after["grafeed_confirmed"], after["additional_info"], after["grafeed_approver"])
    print()     

In [47]:
for graphic in db_client["tweets_pipeline_v2"].find(
    {"$or": [
            {"status":"validated", "source": "dailyshot"},
            {"status":"validated", "source": "twitter", "username": "soberlook"},
        ]},
                                        ):
#         print(graphic["_id"])

    updateAPP(graphic["_id"])







AFTER: daily-shot 2020-09-10 11:21:12.775000 ['commodities', 'energy', 'global', 'oil'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 11:21:12.862000 ['bonds', 'fixed income', 'government bonds', 'fed', 'deficit', 'us', 'funds', 'banking', 'pension funds', 'insurance', 'foreign investment'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 11:21:12.946000 ['personal finance', 'savings', 'us'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 11:21:13.030000 ['government', 'health', 'medicine', 'personal finance', 'social', 'taxes', 'us', 'children', 'labor', 'employment'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 11:21:13.115000 ['futures', 'large caps', 'us', 'robinhood', 'index', 's&p', 'sentiment'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 11:21:13.199000 ['bonds', 'fixed income', 'high yield', 'us'] moderator tool passed bond



In [40]:
for graphic in db_client["tweets_pipeline_v2"].find(    
                                        {"$or": [
                                            {"status":"deleted", "source": "dailyshot"},
                                            {"status":"deleted", "source": "twitter", "username": "soberlook"},
                                            {"status":"trasher", "source": "dailyshot"},            
                                            {"status":"trasher", "source": "twitter", "username": "soberlook"}                                                                
                                                                ]}):
#         print(graphic["_id"])

    updateAPP(graphic["_id"])






AFTER: daily-shot 2020-09-10 11:18:16.965000 ['recession', 's&p', 'developed markets', 'commodities', 'mining', 'us', 'index'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 11:18:17.053000 ['japan', 'pmi', 'index'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 11:18:17.138000 ['transportation', 'forward air', 'logistics', 's&p', 'large caps', 'index'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 11:18:17.222000 ['funds'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 11:18:17.310000 ['us', 'taxes', 'personal finance'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 11:18:17.395000 ['banking', 'us', 'loan', 'credit'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 11:18:17.479000 ['health', 'labor', 'government', 'employment', 'household', 'personal finance', 'loan'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 11:18:17.563000 ['macro', 'us', 'index'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 11:18:

AFTER: daily-shot 2020-09-10 11:18:24.006000 ['s&p', 'index', 'large caps', 'dow jones'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 11:18:24.091000 ['fixed income', 'government bonds', 'macro', 'germany', 'ecb', 'bonds'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 11:18:24.176000 ['futures', 'index', 's&p', 'nasdaq'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 11:18:24.261000 ['ecuador', 'passage bio', 'myanmar', 'taiwan', 'thailand', 'latin america', 'china', 'pharmaceutical industry', 'drugs', 'medicine', 'health', 'india', 'disease', 'bangladesh', 'oil', 'transportation', 'gas', 'commodities', 'global', 'nigeria', 'vietnam', 'peru', 'africa', 'asia', 'philippines', 'cameroon', 'biotech', 'indonesia', 'imperial oil', 'colombia', 'brazil', 'amazon'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 11:18:24.346000 ['fixed income', 'health', 'eu', 'real estate', 'utilities', 'manufacturing', 'csw industrials', 'consumer staples', 'sentimen

In [35]:
for graphic in db_client["tweets_pipeline_v2"].find({
                                    "status": "pipelined",
                                      "username": "soberlook",
                                      "source": "twitter",     
                                    }):
#     print(graphic)
    updateAPP(graphic["_id"])
#     break

In [12]:
for graphic in db_client["tweets_pipeline_v2"].find({
                                      "status": "pipelined",
                                      'source': 'dailyshot'}):
#     print(graphic)
    updateAPP(graphic["_id"])
#     break

AFTER: daily-shot 2020-09-10 10:08:47.962000 ['sentiment', 'funds'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:08:48.049000 ['index', 'us', 'financials', 'credit'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:08:48.134000 ['us', 'entertainment', 'film industry'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:08:48.220000 ['spain', 'eu', 'pandemic', 'disease', 'macro', 'germany', 'italy', 'france', 'gdp'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:08:48.306000 ['oil', 'commodities', 'fuel', 'gasoline'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:08:48.391000 ['bank of america', "moody's", 's&p', 'commodities', 'fitch', 'banking', 'bonds', 'index', 'credit'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:08:48.476000 ['labor', 'employment', 'gender', 'social', 'woman'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:08:48.562000 ['us', 'commerzbank', 'labor', 'employment'] moderator too

AFTER: daily-shot 2020-09-10 10:08:54.017000 ['labor', 'retail', 'personal finance', 'household', 'china'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:08:54.103000 ['china', 'currency', 'credit'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:08:54.189000 ['uk', 'eu', 'russia', 'foreign investment', 'us', 'germany', 'france'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:08:54.274000 ['social', 'health', 'labor'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:08:54.360000 ['recession'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:08:54.445000 ['bonds', 'credit'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:08:54.531000 ['fed', 'anz', 'debt', 'macro', 'global', 'emerging markets', 'gdp'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:08:54.616000 ['digital entertainment', 'entertainment', 'streaming', 'netflix', 'hulu', 'amazon'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:

AFTER: daily-shot 2020-09-10 10:08:59.457000 ['taxes', 'personal finance'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:08:59.542000 ['macro', 'gdp'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:08:59.627000 ['oecd', 'us', 'real estate'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:08:59.712000 ['china', 'debt', 'emerging markets'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:08:59.797000 [] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:08:59.882000 ['macro', 'china', 'gdp'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:08:59.966000 ['fixed income', 'eu', 'africa', 'latin america', 'emerging markets', 'bonds', 'middle east', 'credit'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:00.051000 ['labor', 'employment', 'macro', 'inflation', 'index', 'cpi', 'gdp'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:00.136000 ['pandemic', 'disease', 'macro', 'hong kong', 'gdp'] m

AFTER: daily-shot 2020-09-10 10:09:04.568000 ['services', 'labor', 'employment', 'jobless claims', 'us', 'travel', 'veganism'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:04.656000 ['nyse', 'debt'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:04.741000 ['spain', 'portugal', 'greece', 'italy', 'credit'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:04.826000 ['china', 'population', 'labor', 'employment'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:04.911000 ['us', 'crisis', 'wealth', 'personal finance'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:04.997000 ['wind power', 'asia', 'solar energy', 'renewables', 'us', 'energy', 'import', 'trade'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:05.082000 ['consumer', 'us', 'personal finance'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:05.167000 ['s&p', 'index', 'large caps'] moderator tool passed bond

AFTER: daily-s

AFTER: daily-shot 2020-09-10 10:09:10.447000 ['banking', 'us'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:10.532000 ['covid-19', 'pandemic', 'government', 'disease', 'coronavirus', 'gender', 'social', 'politics', 'crisis', 'woman'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:10.617000 ['funds', 'fed', 'macro', 'us', 'banking', 'credit'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:10.702000 ['banking', 'financials', 'credit'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:10.787000 ['fixed income', 'fed', 'debt', 'macro', 'government debt', 'gdp'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:10.872000 ['oil', 'gas', 'government', 'commodities', 'suncor energy', 'energy', 'politics'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:10.957000 ['bonds', 'fixed income', 'government bonds'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:11.042000 ['technology', 's&p', 'in

...

AFTER: daily-shot 2020-09-10 10:09:42.370000 ['fixed income', 'japan', 'uk', 'eu', 'government bonds', 'acm research', 'macro', 'germany', 'us', 'global', 'bonds', 'index', 'gdp'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:42.455000 ['household', 'mortgage', 'real estate'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:42.540000 ['spain', 'uk', 'labor', 'imf', 'employment', 'germany', 'us', 'italy', 'france'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:42.624000 ['oil', 'commodities', 'cushing'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:42.709000 ['social', 'us', 'personal finance'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:42.794000 ['us', 'real estate'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:42.879000 ['eu', 'consumer', 'household', 'macro', 'gdp'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:42.964000 ['asia', 'china', 'savings', 'personal financ

AFTER: daily-shot 2020-09-10 10:09:47.738000 ['fed', 's&p', 'macro', 'inflation', 'volatility', 'index', 'cpi'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:47.823000 ['banking', 'bonds', 'us', 'loan'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:47.908000 ['s&p', 'macro', 'large caps', 'inflation', 'index'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:47.993000 ['high yield', 'fixed income', 'oil', 'metals', 'government bonds', 's&p', 'small caps', 'commodities', 'dividends', 'mining', 'global', 'large caps', 'emerging markets', 'gold', 'bonds', 'index'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:48.078000 ['fed', 'usd', 'germany', 'ecb', 'us', 'currency', 'interest rates'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:48.163000 ['fixed income', 'hungary', 'thailand', 'china', 'emerging markets', 'india', 'russia', 'macro', 'israel', 'argentina', 'gdp', 'turkey', 'saudi arabia', 'south africa', 

AFTER: daily-shot 2020-09-10 10:09:53.618000 ['oil', 'commodities', 'nymex'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:53.703000 ['etf', 'index', 'large caps', 's&p'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:53.788000 ['restaurants', 'leisure', 'services', 'retail', 'travel', 'ebay', 'amazon'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:53.873000 ['social', 'china', 'politics'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:53.958000 ['household'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:54.043000 ['funds', 'real estate', 'sunstone hotel investors', 'china', 'global', 'travel'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:54.128000 ['consumer', 'index', 'sentiment'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:54.213000 ['bonds', 'fixed income', 'funds'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:54.298000 [] moderator tool passed b

AFTER: daily-shot 2020-09-10 10:09:58.978000 ['fixed income', 'government bonds', 'usd', 'china', 'us', 'yuan', 'currency', 'bonds'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:59.063000 ['fixed income', 'electricity', 'index', 'cpi', 'funds', 'oecd', 'construction & engineering', 'macro', 'transportation', 'fed', 'government bonds', 'vix', 'global', 'inflation', 'energy', 'pmi', 'infrastructure', 'federal signal', 'calamos dynamic convertible and income fund', 's&p', 'us', 'bonds', 'interest rates'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:59.148000 ['sports'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:59.233000 ['banking', 'interest rates'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:59.318000 ['buyback', 'index', 'large caps', 's&p'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:59.403000 [] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:09:59.488000 ['government', 'macro', 

AFTER: daily-shot 2020-09-10 10:10:04.347000 ['oil', 'gem', 'commodities', 'china', 'fms', 'global', 'emerging markets', 'crisis'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:10:04.432000 ['labor', 'personal finance'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:10:04.517000 ['index', 'labor', 'employment'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:10:04.603000 ['services', 'eu', 'retail', 'inflation', 'energy'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:10:04.687000 ['bitcoin', 'cryptocurrency'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:10:04.772000 ['taxes', 'government', 'macro', 'redfin', 'gdp'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:10:04.857000 ['bank of america', 'bonds', 'emerging markets', 'credit'] moderator tool passed bond

AFTER: daily-shot 2020-09-10 10:10:04.943000 ['labor', 'real estate', 'sunstone hotel investors', 'woman', 'personal finance', 'equality', 'gender', 

In [ ]:
"89bio"

In [86]:
db_client["tweets_pipeline_v2"].find({
                                     "status": "graphicone_search",
                                     'in_app': {"$exists": True},
                                     }).count()

15918

In [35]:
for graphic in db_client["tweets_pipeline_v2"].find({"status": "graphicone_feed"}):
    print(graphic)
#     break

{'_id': ObjectId('5dc4c2565dcf6274b6a44809'), 'tweet_text': "The World's 20 Most Profitable Companies 💵\nhttps://t.co/fY7YXoAixd via @VisualCap #money #business #economy #wealth https://t.co/GPqtI7LVLY", 'hashtags': ['money', 'business', 'economy', 'wealth'], 'tweet_timestamp': datetime.datetime(2019, 10, 27, 16, 2), 'username': 'tamaramccleary', 'user_id': 2307675307, 'user_counters': {'followers_count': 306374, 'friends_count': 215979, 'listed_count': 8590}, 'tweet_counters': {'retweet_count': 11, 'favorite_count': 11}, 'img_urls': 'http://pbs.twimg.com/media/EHz92SGVUAAXDDB.jpg', 'tweet_id': 1188486009236836358, 'downloaded': True, 'download_error': False, 'img_type': 'graphic', 'graph_type': 'other-chart', 'confirmed_after_validate': ['saudi aramco', 'apple', 'icbc', 'samsung', 'china construction bank', 'jpmorgan', 'alphabet', 'bank of america', 'shelll', 'shell', 'wells fargo', 'gazprom', 'facebook', 'intel', 'exxonmobil', 'at&t', 'citi', 'toyota'], 'status': 'graphicone_feed', '

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'_id': ObjectId('5ee548faf33aae4b0ad126d6'), 'tweet_text': 'How to Make Your Electric Car More Than Just a Car- Stretching the Field\n\nhttps://t.co/ymkZ3Zto68 https://t.co/blnCoAkAwp', 'hashtags': [], 'username': 'ronloewen', 'tweet_timestamp': datetime.datetime(2020, 6, 3, 16, 18, 55), 'tweet_id': 1268215596132548608, 'img_urls': 'https://pbs.twimg.com/media/EZmbXSLXkAYc4zY.png', 'TIME_FIELD_FOR_SORTING': datetime.datetime(2020, 6, 3, 16, 18, 55), 'source': 'twitter', 'img_type': 'graphic', 'tech_type': 'norm', 'status': 'graphicone_feed', 'extracted_text': "Stretching the Field The U.S. is expected to have a shrinking share of the global electric vehicle market, as coronavirus catalyzes stimulus in China and Europe. China I Europe U.S. I Rest of world 100% 50 2015 '17 '19 '21 '23 '25 '27 2030 Share of global electric vehicle market", 'bert_tags': ['automotive', 'electric vehicles', 'global', 'vehicles'], 'industries_from_bert': ['Cars clothes and other stuff', 'Your health'], 'shif

In [8]:
"""{'_id': ObjectId('5f2177727eefb30587bfee6b'), 'search_text': 'istar "market share" 2020', 'exec_date': datetime.datetime(2020, 7, 29, 13, 3, 14, 527000), 'img_urls': 'https://s3.mordorintelligence.com/man-portable-military-electronics-market/1580109615005_man-portable-military-electronics-market_Man-portable_Military_Electronics_Market%3A_Revenue_%28%25%29%2C_by_Product%2C_Global%2C_2019.png', 'TIME_FIELD_FOR_SORTING': datetime.datetime(2020, 7, 29, 13, 3, 14, 527000), 'source': 'google', 'img_type': 'graphic', 'tech_type': 'norm', 'status': 'graphicone_feed', 'extracted_text': 'Man-portable Military Electronics Market: Revenue (%), by Product, Global, 2019 Communications ISTAR Command and Control 1 Other Products', 'bert_tags': ['electronics', 'technology'], 'industries_from_bert': ['Networks and communications', 'Technology as you know it'], 'shifts_from_bert': ['Connected world', 'Industry 4.0'], 'trasher': [], 'pipelined_ts': datetime.datetime(2020, 7, 29, 13, 19, 46, 624000), 'aws_answer': [{'Score': 0.5646539330482483, 'Type': 'QUANTITY', 'Text': '%', 'BeginOffset': 51, 'EndOffset': 52}, {'Score': 0.9985367059707642, 'Type': 'DATE', 'Text': '2019', 'BeginOffset': 75, 'EndOffset': 79}, {'Score': 0.6245143413543701, 'Type': 'ORGANIZATION', 'Text': 'ISTAR', 'BeginOffset': 95, 'EndOffset': 100}, {'Score': 0.9461585879325867, 'Type': 'QUANTITY', 'Text': '1 Other', 'BeginOffset': 121, 'EndOffset': 128}], 'entity_prepared': [], 'pipelined_issued': True, 'confirmed_after_validate': ['electronics', 'technology', 'military', 'telecom'], 'validated_timestamp': datetime.datetime(2020, 7, 29, 14, 20, 14, 245000), 'validator_username': 'alyona', 'validated_issued': True, 'approved_timestamp': datetime.datetime(2020, 7, 29, 15, 7, 15, 152000), 'board_id': None, 'description': 'Communication equipment is the largest segment of the man-portable military electronics market, but ISTAR segment serving to coordination of battlefield actions is expected to be a more promising area in the future', 'grafeed_approver': 'alyona', 'grafeed_confirmed': ['electronics', 'technology', 'military', 'telecom', 'global'], 'grafeed_industries': ['Networks and communications', 'Technology as you know it'], 'grafeed_shifts': ['Connected world', 'Industry 4.0', 'Growing threats'], 's3_url': 'https://tv-ml-pipeline.s3.us-east-2.amazonaws.com/graphics/GOOGL203782092777372703990502582976391610376', 'dhash': '710c4d0f054f0c31', 'in_app': 'feed', 'upload_timestamp': datetime.datetime(2020, 7, 29, 19, 59, 55, 754000)}"""

'{\'_id\': ObjectId(\'5f2177727eefb30587bfee6b\'), \'search_text\': \'istar "market share" 2020\', \'exec_date\': datetime.datetime(2020, 7, 29, 13, 3, 14, 527000), \'img_urls\': \'https://s3.mordorintelligence.com/man-portable-military-electronics-market/1580109615005_man-portable-military-electronics-market_Man-portable_Military_Electronics_Market%3A_Revenue_%28%25%29%2C_by_Product%2C_Global%2C_2019.png\', \'TIME_FIELD_FOR_SORTING\': datetime.datetime(2020, 7, 29, 13, 3, 14, 527000), \'source\': \'google\', \'img_type\': \'graphic\', \'tech_type\': \'norm\', \'status\': \'graphicone_feed\', \'extracted_text\': \'Man-portable Military Electronics Market: Revenue (%), by Product, Global, 2019 Communications ISTAR Command and Control 1 Other Products\', \'bert_tags\': [\'electronics\', \'technology\'], \'industries_from_bert\': [\'Networks and communications\', \'Technology as you know it\'], \'shifts_from_bert\': [\'Connected world\', \'Industry 4.0\'], \'trasher\': [], \'pipelined_ts\

In [55]:
tags_set = []
for graphic in db_client["tweets_pipeline_v2"].find({
                                    "status": "daily-shot",    
                                    }):
    tags_set += (list(set(graphic["grafeed_confirmed"])))




In [65]:
df = pd.DataFrame.from_dict(Counter(tags_set).most_common())
df.columns=["tag", 'N graphs']
df.to_excel("daily-shot-tags-stats.xlsx")